# 李顺立\_2018210765\_L7
----
----

## 初识爬虫
+ 导入模块
+ 定义爬虫函数
+ 获得前5页数据
+ 总结
----

### 导入模块

In [1]:
import requests                        #打开链接
from bs4 import BeautifulSoup          #解码链接
from collections import OrderedDict    #特定字典？
from urllib.parse import urlencode     #输入内容转码
import time                            #获取时间
import re
import pandas as pd
import numpy as np

-----
### 定义爬虫函数

In [2]:
cookies = {'Cookie': 'U_TRS1=00000033.b9436f90.5cc8015b.330b77ae; U_TRS2=00000033.b94b6f90.5cc8015b.2a9d7c5c; UOR=,search.sina.com.cn,; SINAGLOBAL=120.131.240.51_1556611421.278342; Apache=120.131.240.51_1556611421.278345; ULV=1556611430463:2:2:2:120.131.240.51_1556611421.278345:1556611421786; SUB=_2AkMrlI5af8PxqwJRmPwczmLqZIt0zwHEieKdyH-BJRMyHRl-yD83qhYAtRB6ABSgtYUzDYlgsXEw4yR2WzbQewJlmx2u; SUBP=0033WrSXqPxfM72-Ws9jqgMF55529P9D9WWXg5-BW.46JqEH9bDTz.m4'}
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36'}
##headers :表示访问者信息
###cookies: 记录临时信息，如密码，操作纪律，操作的连续性，
#证书验证设置为False  verify = False

###反爬虫，爬取多页

##如何解决：第一次爬取的时间，就保存这个cookies

###如果出错了：使用try：catch，即（try，except）


###输出结果：  用\001分隔开的dataframe的数据  ->不要标题？？？？
                ##过程：
                        #1.先爬取首页的数据，
                        #2.再爬取超链接的内容
            
                    ###所以需要进行函数拆分
                
                
                ###time.sleep(2)让代码运行时间暂停2秒  可以让时间为随机数
                
                
                



In [3]:
def sina_search(page=2):
    """Function to get  web list pages for a given keyword and page number.

    Args:
        keywords: manual input.
        page: The page number, default 2.

    Returns:
        newsData: A dataframe of the contents in sina web about keywords.
        
    """

    import time as TM
    
    ##输出如期信息
    print(TM.strftime('现在是北京时间：%Y-%m-%d %A %H:%M:%S',TM.localtime(TM.time())))
    
    ###手动输入需要查询的内容
    key_words = input('请输入新闻关键词（空格分开，回车结束）：') 
    key_word = re.split(r'\s', key_words) #按照空格分开
    
    #初始化时间
    time_begin = TM.time()
    
    k = 0  #初始化数据框的index
    newsData = pd.DataFrame(columns = ['title', 'otherinfo', 'abstract','url', 'content'])
    
    
    ###第一层循环：针对输入的词
    for compRawStr in key_word:
        ###设定time.sleep将爬虫伪装
        TM.sleep(np.random.rand()+1)  #暂停时间为1~2的均匀分布随机数
        
        time_keyword = TM.time()
        # Dealing with character encoding
        comp = compRawStr.encode('gbk')  #解码名字
        d = {'q': comp}
        pname = urlencode(d)  #名称编码
        
        count_info = 0  #记录爬取数据条数
        ###第二层循环：针对每一个词的页数进行循环
        for i in range(1, page+1):  #从第一页开始
            href = 'http://search.sina.com.cn/?%s&c=news&from=index&col=&range=&source=&country=&size=&time=&a=&page=%s&pf=0&ps=0&dpc=1'%(pname, i) # comp -> first %s; page -> 2nd %s; col=1_7 -> financial news in sina
            html = requests.get(href, cookies = cookies, headers = headers) #打开链接  #伪装成真实的访问者
            # Parsing html
            soup = BeautifulSoup(html.content, 'html.parser')  #解码链接
            divs = soup.findAll('div', {"class": "box-result clearfix"})  #解码后找到相应内容
            
            ###第三个循环：对每一页的内容进行循环
            for div in divs:  #找到各个文件下的东西
                head = div.findAll('h2')[0] #标题和链接
                # News title
                titleinfo = head.find('a')   #进入子目录
                title = titleinfo.get_text() #获得a的内容：即标题
                # News url
                url = titleinfo['href']  #获得a标题链接
                # Other info
                
                otherinfo = head.find('span', {"class": "fgray_time"}).get_text()  #其他信息
                #source, date, time = otherinfo.split()   #空格分隔
               
                # News abstract
                abstract = div.find('p', {"class": "content"}).get_text()   #摘要文本
                
                ##找到链接中的具体内容
                content = ''
                try:
                    content_htlm = requests.get(url)  #打开标题链接
                    content_soup = BeautifulSoup(content_htlm.content, 'html.parser')  #解码链接
                    content_div = content_soup.find('div', {'id': 'artibody'}) #获得内容
                    p_s = content_div.findAll('p')

                    for p in p_s:
                        try:
                            ptext = p.get_text().strip().replace("\n", "")
                            content += ptext
                        except:
                            pass
                except:
                    pass
                
                
                newsData.loc[k,:] = [title, otherinfo, abstract, url, content]    #将数据放在一个数据框中
                
                k += 1 #index更新
                count_info += 1
                
        print('用时：%.4fs，'%(TM.time() - time_keyword), '找到有关【%s】数据%s页，共%s条。'%(compRawStr, page, count_info))
    print('-------'*8, '\n用时：%.4fs共爬取%s条数据。'%(TM.time() - time_begin, k))   
    return newsData





-----
### 获取前5页数据

In [4]:
news_about_finance = sina_search(5)  #找到新浪新闻前5页内容

现在是北京时间：2019-05-02 Thursday 16:09:40
请输入新闻关键词（空格分开，回车结束）：金融事件 什么鬼 五一节 人山人海


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


用时：24.0057s， 找到有关【金融事件】数据5页，共100条。


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


用时：23.7135s， 找到有关【什么鬼】数据5页，共100条。


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


用时：18.5824s， 找到有关【五一节】数据5页，共100条。


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


用时：19.6008s， 找到有关【人山人海】数据5页，共100条。
-------------------------------------------------------- 
用时：92.5975s共爬取400条数据。


In [5]:
#详细数据
news_about_finance.head(20)

,title,otherinfo,abstract,url,content
0,5月A股怎么走？5大券商为你奉献5月10大金股（附股）,新浪财经综合 2019-05-02 15:38:37,公司自产AC-Hib疫苗再注册受阻是暂时性一过性事件 不影响公司未来价值 独家代理4价和9价...,https://finance.sina.com.cn/stock/stocktalk/20...,新浪财经讯 A股迎来五一假期第二天，为此本文将事先为大家奉献部分券商5月策略和10大金股，具...
1,投资路上最应该读的10本书,雪球 2019-05-02 15:17:00,生物学界的达尔文 《聪明的投资者》旨在将格雷厄姆的理念运用于当代金融市场 的同时 保留其原文...,https://finance.sina.com.cn/roll/2019-05-02/do...,投资路上的十本好书推荐。雪球图片来源：财经小报告作者：听风闻茶1、《聪明的投资者》（第四版 ...
2,投资路上最应该读的10本书,界面 2019-05-02 15:17:00,生物学界的达尔文 《聪明的投资者》旨在将格雷厄姆的理念运用于当代金融市场 的同时 保留其原文...,http://cj.sina.com.cn/articles/view/6192937794...,投资路上最应该读的10本书投资路上的十本好书推荐。雪球图片来源：财经小报告作者：听风闻茶1、...
3,650万送女儿进斯坦福，步长制药董事长的“失败投资”,虎嗅APP 2019-05-02 15:15:00,今年3月份 美国爆出了史上规模最大的招生舞弊案 事件涉及耶鲁大学、斯坦福大学、南加州...,http://t.cj.sina.com.cn/articles/view/23572134...,今年3月份，美国爆出了史上规模最大的招生舞弊案，事件涉及耶鲁大学、斯坦福大学、南加州大学、乔...
4,为何有些球队留不住巨星？马刺成反面教材，魔术专为湖人培养中锋,带未来的自己旅行 2019-05-02 14:58:05,能做到这样的球队 可能仅此一家 但是在伦纳德事件中 马刺真的是太不理智了 不说别的 就在刚刚...,http://k.sina.com.cn/article_7005542808_1a1901...,在如今这个NBA里混，你球队只有一个巨星是远远不够的。不信你看看人家勇士，五个全明星板凳上还...
5,从繁荣疯狂到资本寒冬 美国大萧条最完整的形成和破灭,金融界基金 2019-05-02 14:55:02,这个行为刺激了美国信贷的创造 这是典型的央行不可避免创造金融泡沫的行为 由于低利率环...,http://cj.sina.com.cn/articles/view/1901554085...,其实一切都是一种周期。繁荣，泡沫，崩溃，衰退，萧条，这一切如同一个人的生命周期一样，就像一个...
6,金融学习必看电影！！,WEEX一起交易 2019-05-02 14:30:00,idea 毕竟你面临将会是人人人 想象下在家躺着看几部金融大片...,http://k.sina.com.cn/article_5966772357_163a5b...,五一放假如果你是想去景点？这可能不是一个good idea毕竟你面临将会是人人人想象下在家躺...
7,股指期货的复活,证券市场周刊 2019-05-02 14:12:58,它就可以复活了 本刊特约作者 刘文财／文 毫无怀疑 社会事件不是随机游走 而是...,http://cj.sina.com.cn/articles/view/1741543435...,国内外股指期货市场的发展都曾经遭遇过挫折与指责，等到社会了解了衍生品运行的原理和客观真相后，...
8,IS头目视频现身令西方高度紧张 联军誓言继续追捕,参考消息 2019-05-02 14:00:02,以美国为首的联军发誓要将该组织领导人缉拿归案 据英国《金融时报》网站5月1日报道 ...,https://mil.news.sina.com.cn/2019-05-02/doc-ih...,
9,苏州到底有没有互联网？,CSDN 2019-05-02 13:34:56,【CSDN编者按】都说“上有天堂 下有苏杭” 论风景 苏州绝对可以媲美杭州 但...,http://t.cj.sina.com.cn/articles/view/17987772...,作者 | Python猫责编 | 胡巍巍苏州印象说起苏州，你的第一印象是什么？是“上有天堂，...


In [6]:
news_about_finance.tail(20)

,title,otherinfo,abstract,url,content
380,太过分了，竟然被堵在了火车上~,齐鲁晚报今日德州 2019-05-02 11:29:09,]article_adlist-- 今年五一最大的吐槽 不再是...,http://k.sina.com.cn/article_2237901927_8563ac...,今年五一最大的吐槽不再是各大景点人山人海....也不是各大高速正式化身“ 低速 ”人人身着一...
381,天空之镜、风车海洋、紫藤萝花海……你要的五一浪漫都在这里！,Ourobots 2019-05-02 11:25:08,来这里 回归自然与真实 五一假期已进入倒计时,http://k.sina.com.cn/article_2252672173_86450c...,信阳最近的温度是过山车式的浮动，实现了“今天露腰，明天穿袄”。面对即将到来的夏天，小编不禁感...
382,小长假首日客运班车堵路上 旅客站内煎熬等车,陕西都市快报 2019-05-02 11:20:03,在西安城西客运站前的广场上是人山人海 排队购票的队伍更是庞大壮观 为了应对五一小长假的旅客高...,http://v.sina.com.cn/mobile/2019-05-02/detail-...,
383,实拍！到处是人人人从从众……青果巷“太红”啦！请大家错峰前往、提前绕行！,交汇点资讯 2019-05-02 11:20:01,小编有点被吓到了 我的妈呀！ 同在常州 看着这人山人海 ...,http://k.sina.com.cn/article_5675440730_152485...,昨天是青果巷老街开街来首个节假日先给大家看组动图感受下是不是有点眼熟？这就是咱阔别4年许久未...
384,五一当天加班能拿1000元日薪？“五一”加班费到底怎么算…,燕赵晚报 2019-05-02 11:18:40,假期出门到处人山人海 有没有一个远离喧嚣、避开拥挤人潮 又能赚到钱的好去处...,http://k.sina.com.cn/article_2647971081_9dd4d5...,假期出门到处人山人海。有没有一个远离喧嚣、避开拥挤人潮，又能赚到钱的好去处？当然是加班！那么...
385,五一第一天，人人人人人人人人人人人人人人人人人人人人人人人人人,连云港日报 2019-05-02 11:18:18,天气也好得给力 你们出去玩了吗？ 昨天出游的你是不是看了一幅“人山人海”...,http://k.sina.com.cn/article_3483578150_cfa32f...,昨天是五一假期第一天，天气也好得给力。你们出去玩了吗？昨天出游的你是不是看了一幅“人山人海”...
386,五一假期第二天，成都熊猫基地依旧人山人海,成都事儿君 2019-05-02 11:13:57,五一假期第二天 成都熊猫基地依旧人山人海 滚滚内心是绝望的……[允悲][允悲][允悲]#成...,http://v.sina.com.cn/mobile/2019-05-02/detail-...,
387,爱好莱坞也爱华语片，本青神回复折射有趣灵魂,猫眼电影 2019-05-02 11:06:04,他们的回答五花八门：《人在�逋尽贰痘鸸�英雄》《后会无期》《人山人海》《飞驰人生》《活着》《...,http://k.sina.com.cn/article_2611607127_9ba9f6...,人为什么要看电影？电影是什么颜色的？好莱坞让我们看到“超级英雄”，华语电影让我们看到什么……...
388,五一出游究竟靠不靠谱？首日长城游客5.4万，广州塔全是人头,唔哩 2019-05-02 10:48:45,在节假日的时候去旅游的人数就会非常的多了 简直到了人山人海的地步了！ ...,http://mp.sina.cn/article/2019-05-02/detail-i-...,
389,"大型真人秀""假期堵""全国上线！故宫游客人从众，西湖人太多4G网崩溃",青岛交通广播FM897 2019-05-02 10:43:12,故宫……emmmmm 你看这个队……它又长又宽 八达岭长城上人山人海...,http://k.sina.com.cn/article_1931238691_731c5d...,大型真人秀“假期堵”又和大家见面了一样的剧情 不一样的人群一样的日期 不一样的年份五一假期...


----
### 总结
+ 该爬虫函数只能爬取新浪网的新闻内容，对于其他网页的新闻无法获得，必须重新定义函数。
+ 对于新浪网的新闻，如果新闻的主体内容是音频文件，则无法获得相关资源，爬取能力亟待提高。
+ 一定有更好的方法获得数据。< img src='demo.jpg'> 打开本地图片